In [1]:
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
os.listdir("../input")

['SurveySchema.csv', 'freeFormResponses.csv', 'multipleChoiceResponses.csv']

In [3]:
multiple_df = pd.read_csv('../input/multipleChoiceResponses.csv', low_memory=False)
free_df = pd.read_csv('../input/freeFormResponses.csv', low_memory=False)
schema_df = pd.read_csv('../input/SurveySchema.csv', low_memory=False)

In [4]:
print("Multiple choice response - rows: {} columns: {}".format(multiple_df.shape[0], multiple_df.shape[1]))
print("Free form response - rows: {} columns: {}".format(free_df.shape[0], free_df.shape[1]))
print("Survey schema - rows: {} columns: {}".format(schema_df.shape[0], schema_df.shape[1]))

Multiple choice response - rows: 23860 columns: 395
Free form response - rows: 23860 columns: 35
Survey schema - rows: 12 columns: 52


In [5]:
multiple_df.head(3)

,Time from Start to Finish (seconds),Q1,Q1_OTHER_TEXT,Q2,Q3,Q4,Q5,Q6,Q6_OTHER_TEXT,Q7,Q7_OTHER_TEXT,Q8,Q9,Q10,Q11_Part_1,Q11_Part_2,Q11_Part_3,Q11_Part_4,Q11_Part_5,Q11_Part_6,Q11_Part_7,Q11_OTHER_TEXT,Q12_MULTIPLE_CHOICE,Q12_Part_1_TEXT,Q12_Part_2_TEXT,Q12_Part_3_TEXT,Q12_Part_4_TEXT,Q12_Part_5_TEXT,Q12_OTHER_TEXT,Q13_Part_1,Q13_Part_2,Q13_Part_3,Q13_Part_4,Q13_Part_5,Q13_Part_6,Q13_Part_7,Q13_Part_8,Q13_Part_9,Q13_Part_10,Q13_Part_11,...,Q46,Q47_Part_1,Q47_Part_2,Q47_Part_3,Q47_Part_4,Q47_Part_5,Q47_Part_6,Q47_Part_7,Q47_Part_8,Q47_Part_9,Q47_Part_10,Q47_Part_11,Q47_Part_12,Q47_Part_13,Q47_Part_14,Q47_Part_15,Q47_Part_16,Q48,Q49_Part_1,Q49_Part_2,Q49_Part_3,Q49_Part_4,Q49_Part_5,Q49_Part_6,Q49_Part_7,Q49_Part_8,Q49_Part_9,Q49_Part_10,Q49_Part_11,Q49_Part_12,Q49_OTHER_TEXT,Q50_Part_1,Q50_Part_2,Q50_Part_3,Q50_Part_4,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8,Q50_OTHER_TEXT
0,Duration (in seconds),What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,What is your age (# years)?,In which country do you currently reside?,What is the highest level of formal education ...,Which best describes your undergraduate major?...,Select the title most similar to your current ...,Select the title most similar to your current ...,In what industry is your current employer/cont...,In what industry is your current employer/cont...,How many years of experience do you have in yo...,What is your current yearly compensation (appr...,Does your current employer incorporate machine...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,Select any activities that make up an importan...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,Which of the following integrated development ...,Which of the following integrated development ...,Which of the following integrated development ...,Which of the following integrated development ...,Which of the following integrated development ...,Which of the following integrated development ...,Which of the following integrated development ...,Which of the following integrated development ...,Which of the following integrated development ...,Which of the following integrated development ...,Which of the following integrated development ...,...,Approximately what percent of your data projec...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,What methods do you prefer for explaining and/...,"Do you consider ML models to be ""black boxes"" ...",What tools and methods do you use to make your...,What tools and methods do you use to make your...,What tools and methods do you use to make your...,What tools and methods do you use to make your...,What tools and methods do you use to make your...,What tools and method

In [6]:
free_df.head(3)

,Q11_OTHER_TEXT,Q12_OTHER_TEXT,Q12_Part_1_TEXT,Q12_Part_2_TEXT,Q12_Part_3_TEXT,Q12_Part_4_TEXT,Q12_Part_5_TEXT,Q13_OTHER_TEXT,Q14_OTHER_TEXT,Q15_OTHER_TEXT,Q16_OTHER_TEXT,Q17_OTHER_TEXT,Q18_OTHER_TEXT,Q19_OTHER_TEXT,Q1_OTHER_TEXT,Q20_OTHER_TEXT,Q21_OTHER_TEXT,Q22_OTHER_TEXT,Q27_OTHER_TEXT,Q28_OTHER_TEXT,Q29_OTHER_TEXT,Q30_OTHER_TEXT,Q31_OTHER_TEXT,Q32_OTHER,Q33_OTHER_TEXT,Q34_OTHER_TEXT,Q35_OTHER_TEXT,Q36_OTHER_TEXT,Q37_OTHER_TEXT,Q38_OTHER_TEXT,Q42_OTHER_TEXT,Q49_OTHER_TEXT,Q50_OTHER_TEXT,Q6_OTHER_TEXT,Q7_OTHER_TEXT
0,Select any activities that make up an importan...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,What is the primary tool that you use at work ...,Which of the following integrated development ...,Which of the following hosted notebooks have y...,Which of the following cloud computing service...,What programming languages do you use on a reg...,What specific programming language do you use ...,What programming language would you recommend ...,What machine learning frameworks have you used...,What is your gender? - Prefer to self-describe...,Of the choices that you selected in the previo...,What data visualization libraries or tools hav...,Of the choices that you selected in the previo...,Which of the following cloud computing product...,Which of the following machine learning produc...,Which of the following relational database pro...,Which of the following big data and analytics ...,Which types of data do you currently interact ...,What is the type of data that you currently in...,Where do you find public datasets? (Select all...,During a typical data science project at work ...,What percentage of your current machine learni...,On which online platforms have you begun or co...,On which online platform have you spent the mo...,Who/what are your favorite media sources that ...,What metrics do you or your organization use t...,What tools and methods do you use to make your...,What barriers prevent you from making your wor...,Select the title most similar to your current ...,In what industry is your current employer/cont...
1,NaN,NaN,NaN,NaN,NaN,"Jupyter Notebooks, Pycharm, Intelijidea",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,mlcourse.ai,NaN,ods.ai,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [8]:
df = missing_data(multiple_df)

In [9]:
def plot_percent_of_available_data(title):
    trace = go.Box(
        x = df['Percent'],
        name="Percent",
         marker=dict(
                    color='rgba(238,23,11,0.5)',
                    line=dict(
                        color='tomato',
                        width=0.9),
                ),
         orientation='h')
    data = [trace]
    layout = dict(title = 'Percent of available data  - all columns ({})'.format(title),
              xaxis = dict(title = 'Percent', showticklabels=True), 
              yaxis = dict(title = 'All columns'),
              hovermode = 'closest',
             )
    fig = dict(data=data, layout=layout)
    iplot(fig, filename='percent')

In [10]:
plot_percent_of_available_data('multiple_df')

In [11]:
plot_percent_of_available_data('free_df')

In [12]:
tmp = pd.DataFrame(multiple_df.columns.values)
columns = []
for i in range(1,50):
    var = "Q{}".format(i)
    l = len(list(tmp[tmp[0].str.contains(var)][0]))
    if(l == 1):
        columns.append(var)

print("The columns with only one item in the column group are:\n",columns)

The columns with only one item in the column group are:
 ['Q8', 'Q9', 'Q10', 'Q23', 'Q24', 'Q25', 'Q26', 'Q40', 'Q43', 'Q46', 'Q48']


In [13]:
def get_categories(data, val):
    tmp = data[1::][val].value_counts()
    return pd.DataFrame(data={'Number': tmp.values}, index=tmp.index).reset_index()

In [14]:
df = get_categories(multiple_df, 'Q1')

In [15]:
def draw_trace_bar(data, title, xlab, ylab,color='Blue'):
    trace = go.Bar(
            x = data['index'],
            y = data['Number'],
            marker=dict(color=color),
            text=data['index']
        )
    data = [trace]

    layout = dict(title = title,
              xaxis = dict(title = xlab, showticklabels=True, tickangle=15,
                          tickfont=dict(
                            size=9,
                            color='black'),), 
              yaxis = dict(title = ylab),
              hovermode = 'closest'
             )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='draw_trace')

In [16]:
draw_trace_bar(df, 'Number of people', 'Gender', 'Number of people' )

In [17]:
draw_trace_bar(get_categories(multiple_df,'Q2'), "Number of people in each age range", "Age range", "Number of people", "Purple")

In [18]:
df = get_categories(multiple_df, 'Q3')
df.head()

,index,Number
0,United States of America,4716
1,India,4417
2,China,1644
3,Other,1036
4,Russia,879


In [19]:
trace = go.Choropleth(
            locations = df['index'],
            locationmode='country names',
            z = df['Number'],
            text = df['index'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'rainbow',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Respondents',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Number of respondents per country',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [20]:
draw_trace_bar(get_categories(multiple_df,'Q4'), "Highest level of formal education", "Education", "Number of people", "Magenta")

In [21]:
draw_trace_bar(get_categories(multiple_df,'Q5'), "Undergraduate major (best descrition)", "Undergraduate major", "Number of respondents", "Green")

In [22]:
draw_trace_bar(get_categories(multiple_df,'Q6'), "Current title", "Current title", "Number of respondents", "Red")

In [23]:
draw_trace_bar(get_categories(multiple_df,'Q7'), "Current employer industry", "Current employer industry", "Number of respondents", "Tomato")

In [24]:
draw_trace_bar(get_categories(multiple_df,'Q8'), "Years of experience in the current employer industry", "Years of experience", "Number of respondents", "Lightblue")

In [25]:
draw_trace_bar(get_categories(multiple_df,'Q9'), "Current yearly compensation", "Current yearly compensation", "Number of respondents", "Gold")

In [26]:
draw_trace_bar(get_categories(multiple_df,'Q10'), "Does the current employer uses machine learning", "Use of machine learning", "Number of respondents", "Brown")

In [27]:
draw_trace_bar(get_categories(multiple_df,'Q23'), multiple_df['Q23'][0], "Option", "Number of respondents", "Yellow")

In [28]:
draw_trace_bar(get_categories(multiple_df,'Q24'), multiple_df['Q24'][0], "Option", "Number of respondents", "Lightgreen")

In [29]:
draw_trace_bar(get_categories(multiple_df,'Q25'), multiple_df['Q25'][0], "Option", "Number of respondents", "Dark blue")

In [30]:
draw_trace_bar(get_categories(multiple_df,'Q26'), multiple_df['Q26'][0], "Option", "Number of respondents", "Lightblue")

In [31]:
draw_trace_bar(get_categories(multiple_df,'Q40'), multiple_df['Q40'][0], "Option", "Number of respondents", "Magenta")

In [32]:
draw_trace_bar(get_categories(multiple_df,'Q43'), multiple_df['Q43'][0], "Option", "Number of respondents", "Tomato")

In [33]:
draw_trace_bar(get_categories(multiple_df,'Q46'), multiple_df['Q46'][0], "Option", "Number of respondents", "Green")

In [34]:
draw_trace_bar(get_categories(multiple_df,'Q48'), "Are ML models `black boxes`?", "Are ML models `black boxes`?", "Number of respondents", "Black")

In [35]:
def get_categories_group(data, val_group, val):
    tmp = data[1::].groupby(val_group)[val].value_counts()
    return pd.DataFrame(data={'Number': tmp.values}, index=tmp.index).reset_index()

In [36]:
def draw_trace_group_bar(data_df, val_group, val, title, xlab, ylab,color='Blue'):
    data = list()
    groups = (data_df.groupby([val_group])[val_group].nunique()).index
    for group in groups:
        data_group_df = data_df[data_df[val_group]==group]
        trace = go.Bar(
                x = data_group_df[val],
                y = data_group_df['Number'],
                name = group,
                #marker=dict(color=color),
                text=data_group_df[val]
            )
        data.append(trace)

    layout = dict(title = title,
              xaxis = dict(title = xlab, showticklabels=True, tickangle=15,
                          tickfont=dict(
                            size=9,
                            color='black'),), 
              yaxis = dict(title = ylab),
              hovermode = 'closest'
             )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='draw_trace')

In [37]:
df = get_categories_group(multiple_df, 'Q1', 'Q2')
draw_trace_group_bar(df, 'Q1', 'Q2', 'Number of respondents by Sex and age', 'Age', 'Number of respondents')

In [38]:
df = get_categories_group(multiple_df, 'Q2', 'Q4')
draw_trace_group_bar(df, 'Q2', 'Q4', 'Number of respondents by Age and Highest level of formal education', 'Highest level of formal education', 'Number of respondents')

In [39]:
df = get_categories_group(multiple_df, 'Q2', 'Q9')
draw_trace_group_bar(df, 'Q2', 'Q9', 'Number of respondents by Age and Current yearly compensation', 'Current yearly compensation', 'Number of respondents')

In [40]:
df = get_categories_group(multiple_df, 'Q4', 'Q9')
draw_trace_group_bar(df, 'Q4', 'Q9', 'Number of respondents by Highest level of formal education and Current yearly compensation', 'Current yearly compensation', 'Number of respondents')

In [41]:
df = get_categories_group(multiple_df, 'Q8', 'Q9')
draw_trace_group_bar(df, 'Q8', 'Q9', 'Number of respondents by Years of experience and Current yearly compensation', 'Current yearly compensation', 'Number of respondents')

In [42]:
df = get_categories_group(multiple_df, 'Q6', 'Q4')
draw_trace_group_bar(df, 'Q6', 'Q4', 'Number of respondents by Current title and Highest level of education', 'Current title', 'Number of respondents')